# Pytorch Quantization

## Setup PyTorch

In [19]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib

## Dynamic Quantization


### Load mnist dataset 

In [21]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

### Train model

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.339398
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.433804
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.181050


Train Epoch: 1 [96/60000 (0%)]	Loss: 1.978946
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.057973
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.098002
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.648506
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.759467
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.796752
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.475081
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.509746
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.371608
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.347581
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.242507
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.128363
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.511869
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.314738
Train Epoch: 1 [544/60000 (1%)]	Loss: 1.005181
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.271125
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.338357
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.249766
Train Epoch: 1 [672/60000 (1%)]	Loss: 0.899476
Train Epoch: 1 [704/60000 (1%)]	Loss: 0.991854
Train Epoch: 1 [736/60000 (1%)]	Loss: 0.943413
Train Epoch: 1

### Quantize model

In [23]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

### Check model size

In [24]:
models_dir = pathlib.Path("./models/")
models_dir.mkdir(exist_ok=True, parents=True)
torch.save(model.state_dict(), "./models/original_model.p")
torch.save(quantized_model.state_dict(), "./models/quantized_model.p")

%ls -lh models

total 108K
-rw-rw-r-- 1 harzad harzad 82K Dis  19 22:24 original_model.p
-rw-rw-r-- 1 harzad harzad 23K Dis  19 22:24 quantized_model.p


### Check accuracy

In [25]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return 100. * correct / len(test_loader.dataset)

original_acc = test(model, device, test_loader)
quantized_acc = test(quantized_model, device, test_loader)

print('Original model accuracy: {:.0f}%'.format(original_acc))
print('Quantized model accuracy: {:.0f}%'.format(quantized_acc))

Original model accuracy: 97%
Quantized model accuracy: 97%


## Quantization Aware Training

In [ ]:
qat_model = load_model(saved_model_dir + float_model_file)
qat_model.fuse_model(is_qat=True)

optimizer = torch.optim.SGD(qat_model.parameters(), lr = 0.0001)
# The old 'fbgemm' is still available but 'x86' is the recommended default.
qat_model.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')

torch.ao.quantization.prepare_qat(qat_model, inplace=True)
print('Inverted Residual Block: After preparation for QAT, note fake-quantization modules \n',qat_model.features[1].conv)